In [1]:
using SoftmaxClassifier
using Training
using AbstractTrees

In [2]:
distribution = Dict("A"=>0.4, "B" => 0.2, "C" => 0.2, "D"=> 0.1, "E"=>0.1)

Dict{String,Float64} with 5 entries:
  "B" => 0.2
  "A" => 0.4
  "C" => 0.2
  "D" => 0.1
  "E" => 0.1

In [3]:
classifier_tree,codebook = Training.initialize_network(distribution, 5, WordEmbeddings.huffman_tree)

(Trees.BranchNode with 2 children. data = SoftmaxClassifier.LinearClassifier{2}(5,Float32[-0.0169859 -0.861598; -0.801654 0.604611; -0.473071 -0.406443; 0.0102877 0.877782; 0.128401 -0.114319]),Dict("B"=>[2,1],"A"=>[1],"C"=>[2,2,1],"D"=>[2,2,2,1],"E"=>[2,2,2,2]))

In [ ]:
print_tree(STDOUT, classifier_tree)

In [ ]:
f_A(x) = collect(predict(classifier_tree.data, x))[1]

In [ ]:
@show predict(classifier_tree.data, x)
f_A([0.5f0, 0.5f0, 0.5f0, 0.5f0, 0.5f0])

LoadError: LoadError: UndefVarError: x not defined
while loading In[6], in expression starting on line 200

In [ ]:
using ForwardDiff

In [ ]:
x = [0.5f0, 0.5f0, 0.5f0, 0.5f0, 0.5f0]

@show y=f_A(x)


ForwardDiff.gradient(f_A, x)

y = f_A(x) = 0.34829837f0


5-element Array{Float32,1}:
  0.191716 
 -0.319203 
 -0.0151236
 -0.19691  
  0.0550944

In [ ]:
l_A(x) = log(collect(predict(classifier_tree.data, x))[1])
@time ForwardDiff.gradient(l_A, x; )

  0.033970 seconds (11.48 k allocations: 527.631 KB)


5-element Array{Float32,1}:
  0.550435 
 -0.916465 
 -0.0434214
 -0.565348 
  0.158182 

In [ ]:
l_B(x) = log(collect(predict(classifier_tree.data, x))[2])
ForwardDiff.gradient(l_B, x; )

5-element Array{Float32,1}:
 -0.294177 
  0.4898   
  0.0232063
  0.302147 
 -0.0845393

In [ ]:
function calculate_input_gradient!{F1<:Number, F2<:Number,K}(
                    c::LinearClassifier{K},
                    x::AbstractVector{F1},
                    y::Int64,
                    input_gradient::AbstractVector{F2})
    #input_gradient += -ForwardDiff.gradient(log(collect(predict(c, x))[y]), x)
    outputs = collect(predict(c, x))
    outputs[y] -= 1

    for i in 1:K
        m = outputs[i]
        for j in 1:c.n
            input_gradient[j] -= outputs[i] * c.weights[j, i]
        end
    end
    input_gradient
end


calculate_input_gradient! (generic function with 1 method)

In [ ]:
using Optim

In [ ]:
using WordEmbeddings
using Query
using JLD

In [ ]:
ee = load("../eval/models/plain/tokenised_lowercase_WestburyLab.wikicorp.201004_50__i1.jld","ee");
#ee = restore("../eval/models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model")
#dtree,labels = nn_using AbstractTreestree(ee)
#""

Word embedding(dimension = 50)of 88262 words


In [ ]:
reload("Query")

In [ ]:
function get_simp_optimisable(target_word, embed)
    function loss{F}(x::Vector{F})
        total_lp = zero(F)
        outputs = Vector{F}(2)
        word_lp  = zero(F)
        node = embed.classification_tree
        for y in embed.codebook[target_word]
            c = node.data
            outputs[1],outputs[2] = predict(c, x)
            lp = log(outputs[y])
            @inbounds node = node.children[y]               
            word_lp += lp
        end
        total_lp += word_lp
        total_lp  
    end
    
    @assert(size(ee.classification_tree.data.weights,2) == 2) #Only dealing with binary trees
    function loss_and_grad!{F}(x::Vector{F}, total_input_gradient)
        #Wrong
        total_lp = zero(F)
        outputs = Vector{F}(2)
        input_gradient = similar(x)
        
        word_lp  = zero(F)
        fill!(input_gradient,0.0)

        node = embed.classification_tree
        for y in embed.codebook[target_word]
            @show y
            c = node.data
            outputs[1],outputs[2] = predict(c, x)
            lp = log(outputs[y])

            outputs[y] -= 1  
            
            for j in 1:c.n
                input_gradient[1] -= outputs[1] * c.weights[j, 1]#/exp(lp)
                input_gradient[2] -= outputs[2] * c.weights[j, 2]#/exp(lp)
            end
            @inbounds node = node.children[y]               
            word_lp += lp
        end
        total_lp += word_lp
        total_input_gradient += input_gradient#./exp(word_lp)

        total_lp  
    end
    
    DifferentiableFunction(loss, loss_and_grad!, loss_and_grad!)
end

get_simp_optimisable (generic function with 1 method)

In [ ]:
for i in 1:K
    for j in 1:c.n
        input_gradient[j] -= outputs[i] * c.weights[j, i]
    end
end

LoadError: LoadError: UndefVarError: K not defined
while loading In[17], in expression starting on line 1

In [66]:
function get_optimisable(context, embed)
    function f{F}(x::Vector{F})
        total_lp = zero(F)
        outputs = Vector{F}(2)
        input_gradient = similar(x)
        for target_word in context
            !haskey(embed.codebook, target_word) && continue          
            word_lp  = zero(F)
            node = embed.classification_tree
            for y in embed.codebook[target_word]
                c = node.data
                outputs[1],outputs[2] = predict(c, x)
                lp = log(outputs[y])
                @inbounds node = node.children[y]               
                word_lp += lp
            end
            total_lp += word_lp
        end
        -total_lp  
    end
    
    @assert(size(ee.classification_tree.data.weights,2) == 2) #Only dealing with binary trees
    function fg!{F}(x::Vector{F}, input_gradient)
        fill!(input_gradient, zero(F))
        total_lp = zero(F)
        outputs = Vector{F}(2)
        for target_word in context
            !haskey(embed.codebook, target_word) && continue
            
            word_lp  = zero(F)
            
            node = embed.classification_tree
            for y in embed.codebook[target_word]
                c = node.data
                outputs[1],outputs[2] = predict(c, x)
                lp = log(outputs[y])
                
                outputs[y] -= 1                
                for j in 1:c.n
                    input_gradient[j] += outputs[1] * c.weights[j, 1]
                    input_gradient[j] += outputs[2] * c.weights[j, 2]
                end
                @inbounds node = node.children[y]               
                word_lp += lp
            end
            total_lp += word_lp
        end
        -total_lp  
    end
    
    DifferentiableFunction(f, fg!, fg!)
end

get_optimisable (generic function with 1 method)

In [67]:
word = "ordinary"
context = split("the weighted arithmetic mean is similar to an arithmetic mean")
x_outer = ee.embedding[word]
diff_fun = get_optimisable(context, ee)

@show diff_fun.f(x_outer)

grad = similar(x_outer)
diff_fun.g!(x_outer, grad)
grad_error =abs(grad .- ForwardDiff.gradient(diff_fun.f, x_outer)) 
@assert(all(grad_error .< 1e-5), grad_error)


diff_fun.f(x_outer) = 82.16156f0


In [69]:
@time res = optimize(diff_fun, x_outer)
println("\nProb = ", exp(-1*res.f_minimum))
res

  1.092581 seconds (9.94 M allocations: 173.018 MB, 12.88% gc time)

Prob = 1.5999949229541361e-9


Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [0.26443565,-0.5347172, ...]
 * Minimizer: [-463.1336,-316.9553, ...]
 * Minimum: 2.025327e+01
 * Iterations: 42
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 150
 * Gradient Calls: 150

In [33]:
using NBInclude

In [74]:
nbinclude("../eval/SimTests.ipynb")

In [75]:
function find_optimal_wordvector(ee::GenWordEmbedding, context, word)
    diff_fun = get_optimisable(context, ee)
    x = ee.embedding[word]
    res_status = optimize(diff_fun, x)
    x
end

find_optimal_wordvector (generic function with 2 methods)

In [76]:
wordpairs,contexts,groundsim = load_SCWS("../eval/data/corpora/SCWS/ratings.txt");

In [ ]:
local_sims = get_sims(ee, wordpairs, contexts, find_optimal_wordvector)

pair 1 / 2003
pair 2 / 2003
pair 3 / 2003
2016-08-02T14:37:54.73 - warn: KeyError: key "aglow" not found prefix: "error: "
pair 4 / 2003
pair 5 / 2003
pair 6 / 2003
pair 7 / 2003
pair 8 / 2003
pair 9 / 2003
pair 10 / 2003
pair 11 / 2003
pair 12 / 2003


In [81]:
corspearman(groundsim,local_sims)

0.5086699404515551